# **ArtLine**
**Create** **amazing** **lineart** **portraits** 


In [ ]:
import torch
import torch.nn as nn
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *
from torchvision.models import vgg16_bn
from fastai.utils.mem import *
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torchvision.transforms as transforms

##**Edge Detection**

In [ ]:
def _gradient_img(img):
    img = img.squeeze(0)
    ten=torch.unbind(img)
    x=ten[0].unsqueeze(0).unsqueeze(0)
    
    a=np.array([[1, 0, -1],[2,0,-2],[1,0,-1]])
    conv1=nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1, bias=False)
    conv1.weight=nn.Parameter(torch.from_numpy(a).float().unsqueeze(0).unsqueeze(0))
    G_x=conv1(Variable(x)).data.view(1,x.shape[2],x.shape[3])

    b=np.array([[1, 2, 1],[0,0,0],[-1,-2,-1]])
    conv2=nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1, bias=False)
    conv2.weight=nn.Parameter(torch.from_numpy(b).float().unsqueeze(0).unsqueeze(0))
    G_y=conv2(Variable(x)).data.view(1,x.shape[2],x.shape[3])

    G=torch.sqrt(torch.pow(G_x,2)+ torch.pow(G_y,2))
    return G

gradient = TfmPixel(_gradient_img)

**PATH**

In [ ]:
path = Path('/content/gdrive/My Drive/Apdrawing')

# Blended Facial Features

path_hr = Path('/content/gdrive/My Drive/Apdrawing/draw tiny')
path_lr = Path('/content/gdrive/My Drive/Apdrawing/Tiny Real')

# Portrait Pair

path_hr3 = Path('/content/gdrive/My Drive/Apdrawing/drawing')
path_lr3= Path('/content/gdrive/My Drive/Apdrawing/Real')


**Architecture**

In [ ]:
arch = models.resnet34

### **Facial Features**

In [ ]:
src = ImageImageList.from_folder(path_lr).split_by_rand_pct(0.3, seed=42)


In [ ]:
def get_data(bs,size):
    data = (src.label_from_func(lambda x: path_hr/x.name)
           .transform(get_transforms(xtra_tfms=[gradient()]), size=size, tfm_y=True)
           .databunch(bs=bs,num_workers = 0).normalize(imagenet_stats, do_y=True))

    data.c = 3
    return data

**64px**

In [ ]:
bs,size=20,64

In [ ]:
data = get_data(bs,size)


In [ ]:
data.show_batch(ds_type=DatasetType.Valid, rows=2, figsize=(9,9))

In [ ]:
t = data.valid_ds[0][1].data
t = torch.stack([t,t])

In [ ]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [ ]:
gram_matrix(t)


In [ ]:
base_loss = F.l1_loss


In [ ]:
vgg_m = vgg16_bn(True).features.cuda().eval()
requires_grad(vgg_m, False)

In [ ]:
blocks = [i-1 for i,o in enumerate(children(vgg_m)) if isinstance(o,nn.MaxPool2d)]
blocks, [vgg_m[i] for i in blocks]

In [ ]:
class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): self.hooks.remove()

In [ ]:
feat_loss = FeatureLoss(vgg_m, blocks[2:5], [5,15,2])


In [ ]:
wd = 1e-3
y_range = (-3.,3.)

In [ ]:
def create_gen_learner():
    return unet_learner(data, arch, wd=wd, blur=True,norm_type=NormType.Spectral,self_attention=True, y_range=(-3.0, 3.0),
                        loss_func=feat_loss, callback_fns=LossMetrics)
gc.collect();

In [ ]:
learn_gen = create_gen_learner()


In [ ]:
learn_gen.lr_find()

In [ ]:
lr = 1-01
epoch = 5
def do_fit(save_name, lrs=slice(lr), pct_start=0.9):
    learn_gen.fit_one_cycle(epoch, lrs, pct_start=pct_start,)
    learn_gen.save(save_name)
    learn_gen.show_results(rows=1, imgsize=5)

In [ ]:
do_fit('da', slice(lr))
#lr*10

In [ ]:
learn_gen.unfreeze()


In [ ]:
learn_gen.lr_find()

In [ ]:
epoch = 5
do_fit('db', slice(1E-2))

**128px**

In [ ]:
data = get_data(8,128)
learn_gen.data = data
learn_gen.freeze()
gc.collect()
learn_gen.load('db');

In [ ]:
epoch =5
lr = 1E-03
do_fit('db2',slice(lr))

In [ ]:
learn_gen.unfreeze()


In [ ]:
epoch = 5
do_fit('db3', slice(1e-02,1e-5), pct_start=0.3)

**192px**


In [ ]:
data = get_data(5,192)
learn_gen.data = data
learn_gen.freeze()
gc.collect()
learn_gen.load('db3');

In [ ]:
epoch =5
lr = 1E-06
do_fit('db4')

In [ ]:
learn_gen.unfreeze()


In [ ]:
epoch = 5
do_fit('db5', slice(1e-06,1e-4), pct_start=0.3)

# **Portraits**

In [ ]:
src = ImageImageList.from_folder(path_lr3).split_by_rand_pct(0.2, seed=42)


In [ ]:
def get_data(bs,size):
    data = (src.label_from_func(lambda x: path_hr3/x.name)
           .transform(get_transforms(max_zoom=2.), size=size, tfm_y=True)
           .databunch(bs=bs,num_workers = 0).normalize(imagenet_stats, do_y=True))

    data.c = 3
    return data

**128px**

In [ ]:
data = get_data(8,128)
learn_gen.data = data
learn_gen.freeze()
gc.collect()
learn_gen.load('db5');

In [ ]:
data.show_batch(ds_type=DatasetType.Valid, rows=2, figsize=(9,9))

In [ ]:
learn_gen.lr_find()

In [ ]:
epoch = 5
lr = 1e-03
do_fit('db6')

In [ ]:
learn_gen.unfreeze()


In [ ]:
epoch = 5
do_fit('db7', slice(6.31E-07,1e-5), pct_start=0.3)

**192px**

In [ ]:
data = get_data(4,192)
learn_gen.data = data
learn_gen.freeze()
gc.collect()
learn_gen.load('db7');

In [ ]:
learn_gen.lr_find()

In [ ]:
epoch = 5
lr = 4.37E-05
do_fit('db8')

In [ ]:
learn_gen.unfreeze()


In [ ]:
epoch = 5
do_fit('db9', slice(1.00E-05,1e-3), pct_start=0.3)